In [1]:
from math import pi, sqrt
import pandas as pd
from bokeh.plotting import output_notebook, figure, show
from bokeh.palettes import Category20c, Spectral6
from bokeh.transform import cumsum
output_notebook()

Loading BokehJS ...

In [2]:
million = 1000000
schulden_ende_2016 = 19 * million # laut antrag d. grünen
geldregen_2017 = 382.7 * million
geldregen_2018_erwartet = 89.9 * million + 167 * million
umfang_haushalt_2018 = 3215.894328 * million

In [3]:
def radius_relative(x):
    # die flächen der kreise sollen im richtigen verhältnis stehen
    faktor = x / geldregen_2017  # = (r^2 * pi) / (R^2 * pi) = r^2 / R^2 = (r/R)^2
    r = sqrt(faktor)
    return r

In [4]:
palette = Category20c[20]

kategorien = {"Wohnen":            palette[2],
              "Straßenbau":        palette[1],
              "Krankenhaus":       palette[3],
              "Energiesparen" :    palette[9], 
              "ÖPNV":              palette[10], 
              "Fuß- & Radverkehr": palette[11], 
              "Kultur" :           palette[5], 
              "Bäder" :            palette[6], 
              "Soziales" :         palette[7],
              "Schuldentilgung" :  palette[13],
}

scale = 0.6

def kuchen(x):
    data = pd.DataFrame(x).T.rename(columns={'index':'Zweck'}).rename(columns={0:'value'}).rename(columns={1:'color'})
    Summe = data['value'].sum()
    data['angle'] = data['value']/Summe * 2*pi
    data["color"].replace(kategorien, inplace=True)
    
    scale = 0.6
    
    p = figure(plot_height=500, title="Pie Chart", toolbar_location="above",
               tools="hover", tooltips="@index: @value{($ 0.00 a)}")
    
    cdata = pd.DataFrame({"Überschuss 2017": [geldregen_2017] }).T.rename(columns={0:'value'})
    p.circle(x=-1000, y=1, radius=1 * scale, fill_color=None, line_color='red', line_width=5,
            source=cdata)
    
    p.wedge(x=-1000, y=1, radius=radius_relative(Summe) * scale,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", line_alpha=0, fill_color='color', fill_alpha=0.8, source=data)  # legend='Zweck', 

    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None
    
    show(p)

## Haushaltsüberschuss in Stuttgart 2017 und seine Verwendung

In [5]:
p = figure(plot_height=400, title="Pie Chart", toolbar_location="above",
           tools="hover", tooltips="@index: @value{($ 0.00 a)}")
cdata = pd.DataFrame({"Überschuss 2017": [geldregen_2017] }).T.rename(columns={0:'value'})
p.circle(x=0, y=1, radius=1 * scale, fill_color=None, line_color='red', line_width=5,
        source=cdata)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

In [6]:
show(p)

Unverhoffterweise war die Stadt Stuttgart nicht mit 19 Mio. Schulden, sondern mehr als 380 Mio. Überschuss aus 2017 gegangen. 

Angesichts eines bisherigen Haushaltsplanes für 2018 im Umfang von 3.215.894.328 € war das gar nicht so wenig:

In [7]:
small_scale = 0.4
p = figure(plot_height=400, x_range=(-1.5, 2.5), title="Pie Chart", toolbar_location="above",
           tools="hover", tooltips="@index: @value{($ 0.00 a)}")

cdata = pd.DataFrame({"Überschuss 2017": [geldregen_2017] }).T.rename(columns={0:'value'})
p.circle(x=1.7, y=0, radius=1 * small_scale, fill_color=None, line_color='red', line_width=5,
        source=cdata)

R = radius_relative(umfang_haushalt_2018)
cdata = pd.DataFrame({"Umfang Haushaltsplan 2018": [umfang_haushalt_2018] }).T.rename(columns={0:'value'})
p.circle(x=0, y=0, radius=R * small_scale, fill_color='red', source=cdata)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

In [8]:
show(p)

Die Stadträte frohlockten, und brachten innerhalb weniger Wochen Vorschläge ein, was man mit dem Geldsegen anfangen sollte.

### Vorschlag SÖSLinksPluS

In [9]:
Vorschlag_SÖSLinksPluS = {
    'Wohnbau- und Bodenvorratsfonds':           [ 175 * million, "Wohnen"],
    'VVS-365€-Jahrestickets':                   [  50 * million, "ÖPNV"],
    'Stadtbahnen & Liegenschaften SSB':         [  50 * million, "ÖPNV"],
    'KITA-Gebühren (sonst 2019 gestrichen)':    [26.5 * million, "Soziales"],
    'Radentscheid-Umsetzung 2019':              [18.8 * million, "Fuß- & Radverkehr"],
    'Programm Fußgängerfreundliches Stuttgart': [   5 * million, "Fuß- & Radverkehr"],
    'Kultur':                                   [  30 * million, "Kultur"],
    'Bäderbetriebe':                            [   5 * million, "Bäder"],
    'Schuldenausgleich':                        [21.3 * million, "Schuldentilgung"],
}

In [10]:
kuchen(Vorschlag_SÖSLinksPluS)

### Vorschlag CSU, SPD, Freie Wähler, FDP

In [11]:
Vorschlag_CSU = {
    'Wohnraumoffensive Stuttgart':           [150 * million, "Wohnen"],
    'Aufstockung Friedrichswahl':            [ 25 * million, "Straßenbau"],
    'Kulturelle Infrastruktur':              [  0 * million, "Kultur"], # ?? versteh den Satz nicht
    
    # 'Sanierung und Erweiterungsbauten Oper': [167 * million, "Kultur"],  # die aus der Zukunft
}

In [12]:
kuchen(Vorschlag_CSU)

### Vorschlag Grüne

In [13]:
Vorschlag_Grüne = {
    'Rücklage Katharinenhospital':                                   [200 * million, "Krankenhaus"],
    'Aufstockung Friedrichswahl':                                    [ 25 * million, "Straßenbau"],
    'Kulturelle Infrastruktur':                                      [120 * million, "Kultur"],  # ?? versteh den Satz nicht
    # 'Städtischer Klimaschutzfonds': 55 * million, "Bauen & Wohnen"],
        'Aufstockung für städtischen Niedrigenergie-Bau':            [ 25 * million, "Energiesparen"],
        'Niedrigenergie-Bau städtische Unternehmen und Sportvereine': [10 * million, "Energiesparen"],
        'Enegetinsche Bestandssanierung für Privatpersonen':         [ 6 * million, "Energiesparen"],
        'Förderprogramm "Mein-Solar-Dach"':                          [ 6 * million, "Energiesparen"],
        'Förderung Green IT':                                        [ 3 * million, "Energiesparen"],
    
    # 'Sanierung und Erweiterungsbauten Oper': [167 * million, "Kultur"],  # die aus der Zukunft
}

In [14]:
kuchen(Vorschlag_Grüne)

## Und die Entscheidung? 

... die kann man wohl erst zwei Wochen später einem trockenen, versteckten Onlineprotokoll entnehmen...